In [92]:
import requests
import pandas as pd
import json
import numpy as np
UFC_ALL_EVENTS_API = 'http://ufc-data-api.ufc.com/api/v3/us/events'
FIGHTS_API_SUFFIX = '/fights'

In [7]:

fighter1_stats = ['fighter1_first_name', 'fighter1_last_name', 'fighter1_nickname', \
                  'fighter1_wins', 'fighter1_losses', 'fighter1_draws', 'fighter1_averagefighttime', \
                  'fighter1_averagefighttime_seconds', 'fighter1_kdaverage', 'fighter1_slpm','fighter1_strikingaccuracy',\
                  'fighter1_sapm','fighter1_strikingdefense','fighter1_takedownaverage','fighter1_takedownaccuracy',\
                  'fighter1_takedowndefense','fighter1_submissionsaverage', 'fighter1height', 'fighter1weight', 'fighter1reach']
fighter2_stats = ['fighter2_first_name', 'fighter2_last_name', 'fighter2_nickname', \
                  'fighter2_wins', 'fighter2_losses', 'fighter2_draws', 'fighter2_averagefighttime', \
                  'fighter2_averagefighttime_seconds', 'fighter2_kdaverage', 'fighter2_slpm','fighter2_strikingaccuracy',\
                  'fighter2_sapm','fighter2_strikingdefense','fighter2_takedownaverage','fighter2_takedownaccuracy',\
                  'fighter2_takedowndefense','fighter2_submissionsaverage','fighter2height', 'fighter2weight', 'fighter2reach']

# count = 3
res = requests.get(UFC_ALL_EVENTS_API)
all_events = json.loads(res.content)
all_events_df = pd.read_json(json.dumps(all_events))
df = pd.DataFrame()
for event_id in all_events_df['id']:
    res = requests.get(UFC_ALL_EVENTS_API + '/' + (str)(event_id) + FIGHTS_API_SUFFIX)
    data = json.loads(res.content)
    df = df.append(pd.read_json(json.dumps(data)))
#     if count == 0:
#         break
#     else:
#         count = count - 1

fighter1_stats_df = df.loc[:,fighter1_stats]
fighter1_stats_df = fighter1_stats_df.rename(columns={'fighter1_first_name':'fighter_first_name', \
                                                      'fighter1_last_name':'fighter_last_name',\
                                                      'fighter1_nickname':'fighter_nickname', \
                                                      'fighter1_wins':'fighter_wins', \
                                                      'fighter1_losses':'fighter_losses', \
                                                      'fighter1_draws':'fighter_draws',\
                                                      'fighter1_averagefighttime':'fighter_averagefighttime', \
                                                      'fighter1_averagefighttime_seconds':'fighter_averagefighttime_seconds',\
                                                      'fighter1_kdaverage':'fighter_kdaverage', \
                                                      'fighter1_slpm':'fighter_slpm',\
                                                      'fighter1_strikingaccuracy':'fighter_strikingaccuracy',\
                                                      'fighter1_sapm':'fighter_sapm',\
                                                      'fighter1_strikingdefense':'fighter_strikingdefense',\
                                                      'fighter1_takedownaverage':'fighter_takedownaverage',\
                                                      'fighter1_takedownaccuracy':'fighter_takedownaccuracy',\
                                                      'fighter1_takedowndefense':'fighter_takedowndefense',\
                                                      'fighter1_submissionsaverage':'fighter_submissionsaverage',\
                                                      'fighter1height':'fighter_height', \
                                                      'fighter1weight':'fighter_weight', \
                                                      'fighter1reach' :'fighter_reach'})

fighter2_stats_df = df.loc[:, fighter2_stats]
fighter2_stats_df = fighter2_stats_df.rename(columns={'fighter2_first_name':'fighter_first_name', \
                                                      'fighter2_last_name':'fighter_last_name',\
                                                      'fighter2_nickname':'fighter_nickname', \
                                                      'fighter2_wins':'fighter_wins', \
                                                      'fighter2_losses':'fighter_losses', \
                                                      'fighter2_draws':'fighter_draws',\
                                                      'fighter2_averagefighttime':'fighter_averagefighttime', \
                                                      'fighter2_averagefighttime_seconds':'fighter_averagefighttime_seconds',\
                                                      'fighter2_kdaverage':'fighter_kdaverage', \
                                                      'fighter2_slpm':'fighter_slpm',\
                                                      'fighter2_strikingaccuracy':'fighter_strikingaccuracy',\
                                                      'fighter2_sapm':'fighter_sapm',\
                                                      'fighter2_strikingdefense':'fighter_strikingdefense',\
                                                      'fighter2_takedownaverage':'fighter_takedownaverage',\
                                                      'fighter2_takedownaccuracy':'fighter_takedownaccuracy',\
                                                      'fighter2_takedowndefense':'fighter_takedowndefense',\
                                                      'fighter2_submissionsaverage':'fighter_submissionsaverage',\
                                                      'fighter2height':'fighter_height', \
                                                      'fighter2weight':'fighter_weight', \
                                                      'fighter2reach' :'fighter_reach'})

fighter_stats_df = pd.concat([fighter1_stats_df, fighter2_stats_df]).reset_index(drop=True)
# print('Fighter count before dedup:', len(fighter_stats_df.index))
fighter_stats_df_dedup = fighter_stats_df.drop_duplicates(['fighter_first_name', 'fighter_last_name', 'fighter_nickname'], keep='last')
# print('Fighter count after dedup:', len(fighter_stats_df_dedup.index))
total_fights = fighter_stats_df_dedup['fighter_wins'] + \
               fighter_stats_df_dedup['fighter_losses'] +\
               fighter_stats_df_dedup['fighter_draws']
fighter_stats_df_dedup['total_fights'] = total_fights        
fighter_stats_df_dedup['win %'] = (fighter_stats_df_dedup['fighter_wins']) / total_fights
fighter_stats_df_dedup.to_csv('fighters.csv',index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [111]:
import numpy as np
MINIMUM_FIGHTS_TO_HAVE = 10
for weight_class in fighter_stats_df_dedup['fighter_weight'].unique():
    fighters_in_weight_class = fighter_stats_df_dedup[fighter_stats_df_dedup['fighter_weight'] == weight_class]
#     print(fighters_in_weight_class[['fighter_weight', 'win %']])
    # Rank them by win percentage
    fighters_in_weight_class = fighters_in_weight_class[fighters_in_weight_class.total_fights >= MINIMUM_FIGHTS_TO_HAVE]
    fighters_in_weight_class = fighters_in_weight_class.sort_values(by=['win %', 'total_fights'],ascending=[False, False])
    top_n_fighters = fighters_in_weight_class.loc[:, 
        ['fighter_first_name', 'fighter_last_name', 'total_fights', 'fighter_weight','win %']].head()
    print(top_n_fighters.to_csv(index=False))
    print('==================================================================================================================')

fighter_first_name,fighter_last_name,total_fights,fighter_weight,win %
Israel,Adesanya,12.0,185,1.0
Trevin,Giles,11.0,185,1.0
Paulo,Costa,11.0,185,1.0
Oskar,Piechota,12.0,185,0.9166666666666666
Alessio,Di Chirico,12.0,185,0.9166666666666666

fighter_first_name,fighter_last_name,total_fights,fighter_weight,win %
Joanna,Jedrzejczyk,15.0,115,0.9333333333333332
Viviane,Pereira,14.0,115,0.9285714285714286
Polyana,Viana,11.0,115,0.9090909090909092
Alexa,Grasso,11.0,115,0.9090909090909092
Claudia,Gadelha,18.0,115,0.8333333333333334

fighter_first_name,fighter_last_name,total_fights,fighter_weight,win %
Deiveson,Figueiredo,14.0,125,1.0
Naoki,Inoue,11.0,125,1.0
Magomed,Bibulatov,15.0,125,0.9333333333333332
Jarred,Brooks,14.0,125,0.9285714285714286
Joseph,Morales,10.0,125,0.9

fighter_first_name,fighter_last_name,total_fights,fighter_weight,win %
Brian,Ortega,13.0,145,1.0
Cody,Stamann,17.0,145,0.9411764705882352
Zabit,Magomedsharipov,15.0,145,0.9333333333333332
Arnold,Allen,13.0,145,0.9230769230

## Top 10 fighter per weight class

In [112]:
'''
join UFC data with fightmetric data
to get top M fighters in each weight class
'''

MINIMUM_FIGHTS_TO_HAVE = 10
TOP_N = 10

#Loading Data
fighter_stats_fm = pd.read_excel('./fighter_stats.xlsx')
# fighter_stats_fm.head()
fighter_stats_fm.rename(columns={'Ht.': 'height', 'Wt.': 'weight'}, inplace=True)

fighter_stats_df_dedup = pd.read_csv('fighters.csv', encoding='ISO-8859-1')
all_df = pd.merge(fighter_stats_fm, fighter_stats_df_dedup, how='left', left_on = ['First', 'Last'], right_on = ['fighter_first_name', 'fighter_last_name'])
all_df.head()
for weight_class in all_df['weight'].unique():
    fighters_in_weight_class = all_df[all_df['weight'] == weight_class]
#     print(fighters_in_weight_class[['fighter_weight', 'win %']])
    # Rank them by win percentage
    fighters_in_weight_class = fighters_in_weight_class[fighters_in_weight_class.total_fights >= MINIMUM_FIGHTS_TO_HAVE]
    fighters_in_weight_class = fighters_in_weight_class.sort_values(by=['win %', 'total_fights'],ascending=[False, False])
    top_n_fighters = fighters_in_weight_class.loc[:, 
        ['fighter_first_name', 'fighter_last_name', 'total_fights', 'weight','win %']].head(n=TOP_N)
    print(top_n_fighters.to_csv(index=False))
    print('==================================================================================================================')

fighter_first_name,fighter_last_name,total_fights,weight,win %
Khabib,Nurmagomedov,25.0,155 lbs.,1.0
Gregor,Gillespie,11.0,155 lbs.,1.0
Justin,Gaethje,19.0,155 lbs.,0.9473684210526316
Islam,Makhachev,16.0,155 lbs.,0.9375
Alan,Patrick,16.0,155 lbs.,0.9375
James,Vick,14.0,155 lbs.,0.9285714285714286
Michel,Prazeres,26.0,155 lbs.,0.9230769230769232
Vinc,Pichel,12.0,155 lbs.,0.9166666666666666
Rashid,Magomedov,22.0,155 lbs.,0.9090909090909092
Joaquim,Silva,11.0,155 lbs.,0.9090909090909092

fighter_first_name,fighter_last_name,total_fights,weight,win %
Curtis,Blaydes,10.0,265 lbs.,0.9
Tim,Sylvia,30.0,265 lbs.,0.8666666666666667
Shane,Carwin,14.0,265 lbs.,0.8571428571428571
Gan,McGee,14.0,265 lbs.,0.8571428571428571
Sean,McCorkle,12.0,265 lbs.,0.8333333333333334
Justin,Wren,12.0,265 lbs.,0.8333333333333334
Jared,Rosholt,17.0,265 lbs.,0.8235294117647058
Josh,Copeland,11.0,265 lbs.,0.8181818181818182
Soa,Palelei,27.0,265 lbs.,0.8148148148148148
Ben,Rothwell,46.0,265 lbs.,0.7826086956521741

fi

fighter_first_name,fighter_last_name,total_fights,weight,win %

fighter_first_name,fighter_last_name,total_fights,weight,win %
Mike,Russow,18.0,255 lbs.,0.8333333333333334
Travis,Browne,26.0,255 lbs.,0.6923076923076923

fighter_first_name,fighter_last_name,total_fights,weight,win %

fighter_first_name,fighter_last_name,total_fights,weight,win %

fighter_first_name,fighter_last_name,total_fights,weight,win %

fighter_first_name,fighter_last_name,total_fights,weight,win %

fighter_first_name,fighter_last_name,total_fights,weight,win %

fighter_first_name,fighter_last_name,total_fights,weight,win %

fighter_first_name,fighter_last_name,total_fights,weight,win %

fighter_first_name,fighter_last_name,total_fights,weight,win %

fighter_first_name,fighter_last_name,total_fights,weight,win %

fighter_first_name,fighter_last_name,total_fights,weight,win %

fighter_first_name,fighter_last_name,total_fights,weight,win %

fighter_first_name,fighter_last_name,total_fights,weight,win %
Junior,Dos Sa

In [91]:
def get_fighter_stats(all_df, first, last):
    one = all_df[all_df.First.str.match(first)]
    two = one[one.Last.str.match(last)]
    return two

,First,Last,Nickname,height,weight,Reach,Stance,W,L,D,...,fighter_strikingdefense,fighter_takedownaverage,fighter_takedownaccuracy,fighter_takedowndefense,fighter_submissionsaverage,fighter_height,fighter_weight,fighter_reach,total_fights,win %
1717,Conor,McGregor,The Notorious,"5' 9""",170 lbs.,"74.0""",Southpaw,21,3,0,...,57.27,0.92,62.5,73.91,0.0,0.0,155.0,NaN,24.0,0.875


In [96]:
demo_event = requests.get('http://ufc-data-api.ufc.com/api/v3/us/events/642992/fights')
all_fights= json.loads(demo_event.content)

In [97]:
event_fights = pd.read_json(json.dumps(all_fights))

In [110]:
print(event_fights['fighter1_last_name'])

0       Cyborg
1        Edgar
2     O'Malley
3       Struve
4      Zingano
5        Yoder
6      Dariush
7       Dodson
8     Dollaway
9         Pyle
10     Caraway
11     Johnson
Name: fighter1_last_name, dtype: object
